In [95]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [96]:
cancer = load_breast_cancer()
X_cancer = cancer.data
Y_cancer = cancer.target
scalar = MinMaxScaler()
X_cancer = scalar.fit_transform(X_cancer)

In [97]:
abalone = pd.read_csv('abalone.csv')
X_abalone = abalone.iloc[:, :-1]
X_abalone = pd.get_dummies(X_abalone)
Y_abalone = abalone['Rings'].T
X_abalone = scalar.fit_transform(X_abalone)


In [98]:
def model(X, Y, test, iteration, classifier):
    accuracy = np.array([])
    for i in range(iteration):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = test)
        classifier.fit(X_train, Y_train)
        Y_pred = classifier.predict(X_test)
        accuracy = np.append(accuracy, accuracy_score(Y_test, Y_pred)*100)
    
    accuracy = np.mean(accuracy)
    #matrix = confusion_matrix(Y_test, Y_pred)

    return accuracy


In [104]:
classifier = {'NaiveBayes' : GaussianNB(), 'KNN': KNeighborsClassifier(3), 'DecisionTree' : DecisionTreeClassifier()}
dataset = {'Breast_Cancer' : [X_cancer, Y_cancer], 'Abalone' : [X_abalone, Y_abalone]}

df_holdout1 = pd.DataFrame(np.empty([3, 2]), columns=dataset.keys(), index=classifier.keys())
df_holdout2 = pd.DataFrame(np.empty([3, 2]), columns=dataset.keys(), index=classifier.keys())
df_random1 = pd.DataFrame(np.empty([3, 2]), columns=dataset.keys(), index=classifier.keys())
df_random2 = pd.DataFrame(np.empty([3, 2]), columns=dataset.keys(), index=classifier.keys())

test = {0.25 : [df_holdout1, df_random1] , 0.33 : [df_holdout2, df_random2]}



for k_key,k_value in test.items():
    for i_key,i_value in classifier.items():
        for j_key, j_value in dataset.items():
            k_value[0].loc[i_key, j_key] = model(j_value[0], j_value[1], k_key, 1, i_value)
            k_value[1].loc[i_key, j_key] = model(j_value[0], j_value[1], k_key, 10, i_value)
            

In [105]:
df_holdout1

,Breast_Cancer,Abalone
NaiveBayes,93.006993,11.483254
KNN,95.804196,20.861244
DecisionTree,93.706294,19.904306


In [106]:
df_holdout2

,Breast_Cancer,Abalone
NaiveBayes,94.148936,9.499637
KNN,96.276596,22.407542
DecisionTree,91.489362,19.869471


In [107]:
df_random1

,Breast_Cancer,Abalone
NaiveBayes,93.356643,10.066986
KNN,96.783217,21.684211
DecisionTree,93.286713,19.808612


In [108]:
df_random2

,Breast_Cancer,Abalone
NaiveBayes,93.936170,10.297317
KNN,96.063830,21.508339
DecisionTree,92.925532,19.579405
